Script to process ocean modelling data for ismip7

created by ronja.reese@northumbria.ac.uk

In [1]:
import numpy as np
import xarray as xr
import subprocess
import os, sys, shutil
import gsw
from pathlib import Path
from jinja2 import Environment, FileSystemLoader

In [2]:
models = [ 'timmermann'] # , 'mathiot','naughten_ase_01'] 
modes = ['cold', 'warm']
version = 'v2'

In [3]:
working_dir = '/media/NAS2/ISMIP7/Ocean_Modelling_Data/'
data_dir = '/media/NAS2/ISMIP7/'
output_dir = '/media/NAS2/ISMIP7/share_with_modellers/meltmip/Ocean_Modelling_Data/'

In [4]:
# Define helper functions

def get_data_path(model):
	if model=='mathiot':
		return working_dir+"Pierre_Mathiot"
	if model=='timmermann':
		return working_dir+"Ralph_Timmermann/TimmermannUndGoeller2017_V2"
	if model=='naughten_ase_01':
		return working_dir+"Kaitlin_Naughten/MITgcm_ASE/RCP85_ens01"

def get_file_name(model,mode):
	if model=='mathiot' and mode=='cold':	
		return "MATHIOT_REF_TandS_1989-2018_Ant_ISMIP6grid_withtimeaxis_compressed.nc"
	if model=='mathiot' and mode=='warm':	
		return "MATHIOT_WARM_TandS_2048-2098_Ant_ISMIP6grid_withtimeaxis_compressed.nc"
	if model=='timmermann' and mode=='warm':
		return "RANGO.A1B.2000-2199.TS.aym.ismip.91levels.nc"
	if model=='timmermann' and mode=='cold':
		return "RANGO.20C.1950-1999.TS.aym.ismip.91levels.nc"
	if model=='naughten_ase_01':
		return "MITgcm_ASE_RCP85_ens01.nc" # FIXME here: We need to collate Kailtin's datasets into one 

def combine_dataset(model, input_files):
	ds = xr.open_mfdataset(input_files,combine = 'by_coords', concat_dim="time")
	ds.to_netcdf(get_file_name(model, 'cold'))
	
def model_specific_pre_processing(model, input_file, output_file):
	print('Model specific adjustments')
	
	if model=='mathiot':
		old_name = ["votemper", "vosaline"]
		new_name = ["theta_ocean", "salinity_ocean"]
	
	if model=='timmermann':
		old_name = ["anzxismip", "anzyismip", "anzzismip", "anzyears", "t", "temp", "salt"]
		new_name = ["x", "y", "z", "time", "time", "theta_ocean", "salinity_ocean"]

	ds_renamed = xr.open_dataset(input_file)
	for i, old_n in enumerate(old_name):
		ds_renamed = ds_renamed.rename({ old_n: new_name[i]})
	ds_renamed.to_netcdf(output_file)



def further_processing_mathiot(output_file):
	# Add depth dimension
	
	depth_file = os.path.join(dpath,"mask_dim_info_NEMO_ISMIP6_grid.nc")
	
	subprocess.call("ncks -A -v depth %s %s" %(depth_file, output_file),shell=True)
	# Note, this requires module load nco
	subprocess.call("ncap2 -s 'depth=-depth' -O %s %s" %(output_file, output_file),shell=True)
		
	ds_coords = xr.open_dataset(working_dir+"/Pierre_Mathiot/mask_dim_info_NEMO_ISMIP6_grid.nc")

	ds = xr.open_dataset(output_file)
	ds['deptht'] = -1*ds_coords['depth']
		
	target_grid = xr.open_dataset(data_dir+"/ismip_grid/ismip_8km_60m_grid.nc")

	interpolated = ds.interp(deptht=target_grid["z_extrap"], method='linear') 
	ds.close()
		
	interpolated = interpolated.drop_vars(['lat', 'lon', 'deptht'], errors='ignore')
	
	interpolated = interpolated.fillna(-9999)
	encoding = {var: {'_FillValue': -9999} for var in interpolated.data_vars}
	
	
	interpolated.to_netcdf(output_file , encoding=encoding, unlimited_dims='time') # FIXME?
	interpolated.close()
	return 0


def further_processing_timmermann(output_file):
    # Rename z to z_extrap
    #subprocess.call("ncrename -d z,z_extrap -v z,z_extrap  %s" %(output_file),shell=True)
    #return 0
    
    # Add z_extrap variable
    #grid_file = os.path.join(data_dir,"ismip_grid", "ismip_8km_60m_grid.nc")
    #subprocess.call("ncks -A -v z_extrap,x,y %s %s" %(grid_file, output_file),shell=True)

    # interpolate
    ds = xr.open_dataset(output_file)
        
    target_grid = xr.open_dataset(data_dir+"/ismip_grid/ismip_8km_60m_grid.nc")
    
    interpolated = ds.interp(z=target_grid["z_extrap"], method='linear') 
    ds.close()
    #interpolated = interpolated.drop_vars(['lat', 'lon', 'deptht'], errors='ignore')
    
    interpolated = interpolated.fillna(-9999)
    encoding = {var: {'_FillValue': -9999} for var in interpolated.data_vars}
    
    
    interpolated.to_netcdf(output_file , encoding=encoding, unlimited_dims='time') # FIXME?
    interpolated.close()
    return 0
    

def calculate_time_mean(input_file,output_file):
    print('Calculate temporal mean')
    
    ds = xr.load_dataset(input_file)
    thetamean = ds['theta_ocean'].isel(time=slice(-21,-1)).mean('time',keepdims=True,keep_attrs=True)
    salmean = ds['salinity_ocean'].isel(time=slice(-21,-1)).mean('time',keepdims=True,keep_attrs=True)
    timemean = ds['time'].isel(time=slice(-21,-1)).mean('time',keepdims=True,keep_attrs=True)
    
    ds_new = xr.Dataset({ 'theta_ocean' :  thetamean, 'salinity_ocean': salmean, 'time':timemean})
    ds_new.attrs = ds.attrs.copy()
    
    ds_new['theta_ocean'].attrs = ds['theta_ocean'].attrs.copy()
    ds_new['salinity_ocean'].attrs = ds['salinity_ocean'].attrs.copy()
    ds_new['time'].attrs = ds['time'].attrs.copy()
    # Copy encoding (important for _FillValue, dtype, etc.)
    ds_new['theta_ocean'].encoding = ds['theta_ocean'].encoding.copy()
    ds_new['salinity_ocean'].encoding = ds['salinity_ocean'].encoding.copy()
    ds_new['time'].encoding = ds['time'].encoding.copy()
    ds_new = ds_new.fillna(-9999)
    encoding = {var: {'_FillValue': -9999} for var in ds_new.data_vars}
    
    ds_new["time"].attrs["calendar"] = 'proleptic_gregorian'
    ds_new.attrs['history'] = 'test'
    # Then copy to mounted drive
    tmp_file = output_file.split('/')[-1]
    ds_new.to_netcdf(tmp_file, unlimited_dims='time', encoding=encoding)
    shutil.copy(tmp_file, output_file)
    os.remove(tmp_file)  
    return 0
	
	
def remove_ice_shelf_data(input_file,output_file):

	topography = xr.open_dataset(data_dir+"Topography/bedmap3_ismip_8km.nc")
	
	ds = xr.open_dataset(input_file)
	encoding = {} # make sure to keep fill values
	for var in ds.data_vars:
	    if '_FillValue' in ds[var].encoding:
        	encoding[var] = {'_FillValue': ds[var].encoding['_FillValue']}
	ds['theta_ocean'] = ds['theta_ocean'].where(np.logical_and(topography['floating_frac']==0,topography['grounded_frac']==0), np.nan)
	ds['salinity_ocean'] = ds['salinity_ocean'].where(np.logical_and(topography['floating_frac']==0,topography['grounded_frac']==0), np.nan)
	ds.to_netcdf(output_file, encoding=encoding)

	
def regrid_20m_to_60m_vertical(input_file,output_file):
	
    ds = xr.open_dataset(input_file)	
    target_grid = xr.open_dataset(data_dir+"/ismip_grid/ismip_8km_60m_grid.nc")
    
    interpolated = ds.interp(z_extrap=target_grid["z"], method='linear') 
    ds.close()
    interpolated = interpolated.drop_vars(['lat', 'lon', 'z_extrap'], errors='ignore')
    interpolated.to_netcdf(output_file)


def calculate_thermal_forcing(dataset):
	# note that Xylars routine is for SA and CT
    
	dsT = xr.open_dataset(dataset+'_T.nc') # potential temp
	dsS = xr.open_dataset(dataset+'_S.nc') # practical salinity

	pressure = 1028*9.81*dsT['z']*-1
	
	# Linearisation of freezing point for potential temp, practical salinity from Reese et al., 2018
	a = -0.0572 # deg per PSU
	b = 0.0788 # degC
	c = 7.77e-8 # degC per Pa

	pt_freeze = a*dsS['salinity_ocean'] +b - c*pressure
	tf = dsT['theta_ocean'] - pt_freeze
	ds_tf = xr.Dataset({'thermal_forcing': np.squeeze(tf) })
	ds_tf.to_netcdf(dataset+'_TF.nc')



def drop_time_dimension(infile, outfile):
	ds = xr.open_dataset(infile)
	ds = ds.squeeze().drop_vars('time')
	
	if "_S" in infile:
		ds_new = xr.Dataset({'salinity_ocean': ds.salinity_ocean})
	elif "_T" in infile:
		ds_new = xr.Dataset({'theta_ocean': ds.theta_ocean})
	else:
   		print('Potentially wrong file name' )
	ds.close()
	ds_new.to_netcdf(outfile)
	
    

def get_final_name(model, mode, version):
	if model == 'mathiot':
		return 'Mathiot23_'+mode+'_'+version
	elif model == 'timmermann':
		return 'TimmermannUndGoeller2017_'+mode+'_'+version
	else:
		print('something went wrong')


def create_namelist_files(file_in, model, mode, var):

    env = Environment(loader=FileSystemLoader('./'),  trim_blocks=True, lstrip_blocks=True,  keep_trailing_newline=True)
    template = env.get_template('namelisttemplate.nml')

    if var =='S':
        variable = 'salinity_ocean'
    elif var=='T':
        variable = 'theta_ocean'
    else:
        print('Wrong variable')

    if model == 'timmermann':
        z = 'z_extrap'
    elif model == 'mathiot':
        z = 'z_extrap'
    else:
        print('Model not defined yet')
    file_out = os.path.join(output_dir, get_final_name(model, mode,version)+'_'+var+'.nc' )
    file_tmp = 'tmp_'+model+mode+var+'.nc'
    
    namelist_name = 'namelist_'+model+'_'+mode+'_'+var+'.nml'
    
    data_dict = {
        'file_in':file_in,
        'file_out':file_out,
        'file_out_horizontal':file_tmp,
        'file_basin': '/media/NAS2/ISMIP7/imbie2_basins_for_ISMIP7/basinNumbers_8km.nc' ,
        'file_topo': '/media/NAS2/ISMIP7/Topography/bedmap3_ismip_8km.nc',
        'variable':variable,
        'z_name':z,
    }
        
    output = template.render(data_dict).replace('\r\n', '\n')
    with open(namelist_name, 'w', newline='\n') as f:
        f.write(output)


def clean_directory_before_extrapolation():
    cmd = 'rm -rf job* namelist_* output_logs tmp_*.nc'
    subprocess.check_call(cmd,shell=True)
    

In [30]:
#clean_directory_before_extrapolation()

In [13]:
# MAIN - preprocess ocean data 


for model in models:
    for mode in modes:
        print(model, mode)
        
        dpath=get_data_path(model)
        mfile=get_file_name(model,mode)
        
        ## model-data specific pre-processing, do not skip
        if "naughten_ase" in model: # FIXME HERE to try Naughten data processing
            input_files =   f"{Path(os.path.join(dpath,mfile)).stem}_*.nc"
            combine_dataset(model, input_files)
        orig_file = os.path.join(dpath,mfile )
        output_file = os.path.join(dpath, f"{Path(orig_file).stem}_renamed.nc")
        model_specific_pre_processing(model,orig_file ,output_file)
        
        # temporal averaging
        if model=='mathiot' and mode=='cold':
            tap = ['1998', '2018']
        if model=='mathiot' and mode=='warm':
            tap = ['2078', '2098']
        if model=='timmermann' and mode=='cold':
            tap=['1979', '1999']		
        if model=='timmermann' and mode=='warm':
            tap=['2179', '2199']		
        input_file = os.path.join(dpath,f"{Path(orig_file).stem}_renamed.nc")
        timemean_file=os.path.join(dpath,f"{Path(input_file).stem}"+"_"+tap[0]+"_"+tap[1]+".nc")
        calculate_time_mean(input_file,timemean_file)
        
        # further processing mathiot (vertical interpolation onto extrapolation grid)
        if model=='mathiot':
            further_processing_mathiot(timemean_file)
        if model=='timmermann':
            further_processing_timmermann(timemean_file)
            
        # remove data underneath ice shelves for consistency with CMIP forcing
        output_file=os.path.join(dpath, f"{Path(timemean_file).stem}_noshelf.nc")
        remove_ice_shelf_data(os.path.join(dpath,timemean_file), output_file)
    
        
        # Create namelist file file_in, model, mode, var
        create_namelist_files(os.path.join(dpath, output_file), model, mode, 'T')
        create_namelist_files(os.path.join(dpath, output_file), model, mode, 'S')
        
        # add the file to the joblist
        with open('joblist_horizontal.txt', 'a') as fh:
            fh.write("i7aof_extrap_horizontal namelist_"+model+"_"+mode+"_T.nml\n")
            fh.write("i7aof_extrap_horizontal namelist_"+model+"_"+mode+"_S.nml\n")
        
        with open('joblist_vertical.txt', 'a') as fv:
            fv.write("i7aof_extrap_vertical namelist_"+model+"_"+mode+"_T.nml\n")
            fv.write("i7aof_extrap_vertical namelist_"+model+"_"+mode+"_S.nml\n")
            

timmermann cold
Model specific adjustments
Calculate temporal mean
timmermann warm
Model specific adjustments
Calculate temporal mean


In [8]:
# Run extrapolation

# extrapolation horizontal and vertical, make sure to adjust namelist files to correct folders
# !!!! Note that this requires to install and activate the ismip7_dev environment 

cmd = 'parallel  --joblog joblogH.txt --bar --results output_logs --tag -j 8 < joblist_horizontal.txt'
subprocess.check_call(cmd,shell=True)

cmd = 'parallel  --joblog joblogV.txt --bar --results output_logs --tag -j 8 < joblist_vertical.txt'
subprocess.check_call(cmd,shell=True)

In [5]:
# Postprocess ocean modelling data

for model in models:
	for mode in modes:
		print(model, mode)		
		# calculate thermal forcing
		
		finalname= get_final_name(model, mode,version)
		print(finalname)
		
		print('Regrid to 60m vertical')
		input_file = os.path.join(output_dir, finalname+'_S.nc')
		output_file = os.path.join(output_dir, finalname+'_S2.nc')
		regrid_20m_to_60m_vertical(input_file,output_file)
        
		input_file = os.path.join(output_dir, finalname+'_T.nc')
		output_file = os.path.join(output_dir, finalname+'_T2.nc')
		regrid_20m_to_60m_vertical(input_file,output_file)
		
		print('Calculate thermal forcing')
		
		drop_time_dimension(os.path.join(output_dir, finalname+'_S2.nc'), os.path.join(output_dir, finalname+'_S.nc') )
		drop_time_dimension(os.path.join(output_dir, finalname+'_T2.nc'), os.path.join(output_dir, finalname+'_T.nc') )
		calculate_thermal_forcing(os.path.join(output_dir, finalname))

timmermann cold
TimmermannUndGoeller2017_cold_v2
Regrid to 60m vertical
Calculate thermal forcing
timmermann warm
TimmermannUndGoeller2017_warm_v2
Regrid to 60m vertical
Calculate thermal forcing


In [35]:
## clean up
cmd = 'rm -rf '+output_dir+'/*2.nc'
subprocess.check_call(cmd,shell=True)

0